In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('train-data.csv')
data.head(5)

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [3]:
target_column = 'class'
attrs = list(data)
possible_classes = data[target_column].unique()
p_class = {}
for class_name in possible_classes:
    p_class[class_name] = data[target_column].value_counts()[class_name] / len(data)
p_class

{'acc': 0.22222222222222221,
 'good': 0.039930555555555552,
 'unacc': 0.70023148148148151,
 'vgood': 0.037615740740740741}

In [4]:
likehood_table = {}
for class_name in possible_classes:
    for attr in attrs:
        if attr != target_column:
            attr_values = data[attr].unique()
            only_current_class = data[data[target_column] == class_name]
            for attr_value in attr_values:
                key = (class_name, attr, attr_value)
                value = 0
                if attr_value in only_current_class[attr].value_counts():
                    value = only_current_class[attr].value_counts()[attr_value] * 1.0 / len(only_current_class)
                likehood_table[key] = value
likehood_table

{('acc', 'buying', 'high'): 0.28125,
 ('acc', 'buying', 'low'): 0.23177083333333334,
 ('acc', 'buying', 'med'): 0.29947916666666669,
 ('acc', 'buying', 'vhigh'): 0.1875,
 ('acc', 'doors', '2'): 0.2109375,
 ('acc', 'doors', '3'): 0.2578125,
 ('acc', 'doors', '4'): 0.265625,
 ('acc', 'doors', '5more'): 0.265625,
 ('acc', 'lug_boot', 'big'): 0.375,
 ('acc', 'lug_boot', 'med'): 0.3515625,
 ('acc', 'lug_boot', 'small'): 0.2734375,
 ('acc', 'maint', 'high'): 0.2734375,
 ('acc', 'maint', 'low'): 0.23958333333333334,
 ('acc', 'maint', 'med'): 0.29947916666666669,
 ('acc', 'maint', 'vhigh'): 0.1875,
 ('acc', 'persons', '2'): 0,
 ('acc', 'persons', '4'): 0.515625,
 ('acc', 'persons', 'more'): 0.484375,
 ('acc', 'safety', 'high'): 0.53125,
 ('acc', 'safety', 'low'): 0,
 ('acc', 'safety', 'med'): 0.46875,
 ('good', 'buying', 'high'): 0,
 ('good', 'buying', 'low'): 0.66666666666666663,
 ('good', 'buying', 'med'): 0.33333333333333331,
 ('good', 'buying', 'vhigh'): 0,
 ('good', 'doors', '2'): 0.21739

In [5]:
test_data = data.sample(10)
# test_data = test_data.loc[:, test_data.columns != target_column]
predicted = 'predicted'
test_data[predicted] = ''
test_data

,buying,maint,doors,persons,lug_boot,safety,class,predicted
803,high,low,3,more,small,high,acc,
1537,low,med,2,more,big,med,good,
1512,low,med,2,2,small,low,unacc,
1429,low,high,2,more,big,med,acc,
1558,low,med,3,more,small,med,acc,
907,med,vhigh,3,4,big,med,acc,
778,high,low,2,more,med,med,unacc,
861,high,low,5more,more,big,low,unacc,
124,vhigh,high,2,4,big,med,unacc,
355,vhigh,low,3,2,med,med,unacc,


In [6]:
for row in test_data.iterrows():
    possible = {}
    for possible_class in possible_classes:
        possible_value = p_class[possible_class]
        row_attrs = list(test_data)
        for attr in row_attrs:
            if attr != target_column and attr != predicted:
                attr_value = test_data.get_value(row[0], attr)
                key = (possible_class, attr, attr_value)
                value = 0
                if key in likehood_table:
                    value = likehood_table[key]
                possible_value *= value
        possible[possible_class] = possible_value
        
    max_value = 0
    result_class = '-'
    for key, value in possible.items():
        if value > max_value:
            max_value = value
            result_class = key
            
    test_data = test_data.set_value(row[0], predicted, result_class)
    
test_data

,buying,maint,doors,persons,lug_boot,safety,class,predicted
803,high,low,3,more,small,high,acc,acc
1537,low,med,2,more,big,med,good,acc
1512,low,med,2,2,small,low,unacc,unacc
1429,low,high,2,more,big,med,acc,acc
1558,low,med,3,more,small,med,acc,acc
907,med,vhigh,3,4,big,med,acc,acc
778,high,low,2,more,med,med,unacc,unacc
861,high,low,5more,more,big,low,unacc,unacc
124,vhigh,high,2,4,big,med,unacc,unacc
355,vhigh,low,3,2,med,med,unacc,unacc


In [7]:
test_data['result'] = test_data[target_column] == test_data[predicted]
test_data

,buying,maint,doors,persons,lug_boot,safety,class,predicted,result
803,high,low,3,more,small,high,acc,acc,True
1537,low,med,2,more,big,med,good,acc,False
1512,low,med,2,2,small,low,unacc,unacc,True
1429,low,high,2,more,big,med,acc,acc,True
1558,low,med,3,more,small,med,acc,acc,True
907,med,vhigh,3,4,big,med,acc,acc,True
778,high,low,2,more,med,med,unacc,unacc,True
861,high,low,5more,more,big,low,unacc,unacc,True
124,vhigh,high,2,4,big,med,unacc,unacc,True
355,vhigh,low,3,2,med,med,unacc,unacc,True
